In [23]:
from numpy import mean, std

from seances.models import Seance
from sensors.models import SensorRecord, Sensor

SENSORS = ["fsr_01","fsr_02","fsr_03","fsr_04","accel01_x","accel01_y","accel01_z","gyro01_x","gyro01_y","gyro01_z","cpuusage_01","cpuusage_02","cpuusage_03","cpuusage_04","mempercentage_01","netpacketssent_01","netpacketsreceived_01"]

def process_data():
    """
    Shape our data in a way that is usable by the autoencoder.
    """
    data = []
    seances = Seance.objects.filter(valid=True, experiment__sequence_number=1)
    sensors = Sensor.objects.filter(topic__in=SENSORS)

    for seance in seances:
        print(seance)
        row_data = {"user_id": seance.user.id, "seance_id": seance.id}
        valid = True
        for sensor in sensors:
            try:
                sensor_data = to_n_points(
                    [
                        x.value
                        for x in SensorRecord.objects.filter(
                            seance=seance, sensor=sensor
                        )
                    ],
                    50,
                )
            except ValueError:
                print("Missin data in seance... skipping.")
                valid = False
                break
            row_data.update({sensor.topic: sensor_data})
        if valid:
            data.append(row_data)
    return data

def to_n_points(data: list, n: int):
    """
    Take the provided list of values and compress it to a length of n elements.
    This is achieved by averaging elements.
    """
    if len(data) < n:
        raise ValueError("Not enough data to compress to {} elements.".format(n))

    step = len(data) / n
    i = 0
    result = []
    for _ in range(0, n):
        row = data[round(i) : round(i + step)]
        result.append(mean(row))
        i += step
    return result

data = process_data()


Completed seance started at: 2019-09-13 08:44:53 with user test_subject_01
Completed seance started at: 2019-09-18 09:23:40 with user test_subject_02
Completed seance started at: 2019-09-18 09:46:16 with user test_subject_02
Completed seance started at: 2019-09-20 06:24:48 with user test_subject_03
Missin data in seance... skipping.
Completed seance started at: 2019-09-20 10:02:47 with user test_subject_04
Completed seance started at: 2019-09-20 10:29:38 with user test_subject_04
Completed seance started at: 2019-09-23 09:03:16 with user test_subject_05
Completed seance started at: 2019-09-23 09:40:48 with user test_subject_05
Completed seance started at: 2019-09-24 13:04:29 with user test_subject_06
Missin data in seance... skipping.
Completed seance started at: 2019-09-25 07:06:48 with user test_subject_07
Completed seance started at: 2019-09-25 07:52:06 with user test_subject_07
Completed seance started at: 2019-09-25 08:59:18 with user test_subject_08
Completed seance started at: 2

In [21]:
X = []
for row in data:
    x = []
    for sensor in SENSORS:
        x += row[sensor]
    X.append(x)

print(X[0])

[-9.154258801232905e-15, 3.045262423937079e-15, 7.245271757328857e-15, 5.9040802096739285e-15, -2.441727614295877e-15, -6.6117500225815775e-15, 8.260279694878584e-15, 3.03258934369075e-15, 7.616736026013976e-15, 5.113086684129753e-16, 3.050457219982873e-15, 7.951731360491444e-15, -8.181573121803775e-15, 1.897131473146074e-14, -1.1549418954270414e-15, 1.0339967050305661e-16, -2.2274929017945706e-17, 9.171220774672707e-17, 5.4449826488311727e-17, 6.819978267222883e-17, 0.0, -1.2086431880773703e-17, 3.9980813448945706e-17, -6.879968916748108e-17, 4.4629745245334743e-17, 7.624248146078018e-16, 1.1138111840897612e-15, 8.649337607181801e-16, 9.11131018704479e-16, 8.640039743589022e-16, -2.4695125702418558e-15, -1.0454763576869254e-15, -3.5529461253903206e-16, -1.5676334580745816e-15, 5.485739519739062e-17, 8.4378112104461e-17, 2.947229927850203e-16, 1.2459137214322616e-16, 3.421389947788248e-16, -1.1203925629297576e-16, -7.438290874222456e-18, 2.7194471731740832e-17, -9.042172343976674e-17, 

In [ ]:
from keras.layers import Input, Dense
from keras.models import Model

# this is the size of our encoded representations
encoding_dim = 16  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

# this is our input placeholder
input_img = Input(shape=(784,))
# "encoded" is the encoded representation of the input
encoded = Dense(encoding_dim, activation="relu")(input_img)
# "decoded" is the lossy reconstruction of the input
decoded = Dense(784, activation="sigmoid")(encoded)

# this model maps an input to its reconstruction
autoencoder = Model(input_img, decoded)

# this model maps an input to its encoded representation
encoder = Model(input_img, encoded)

# create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim,))
# retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# create the decoder model
decoder = Model(encoded_input, decoder_layer(encoded_input))

autoencoder.compile(optimizer="adadelta", loss="binary_crossentropy")

from keras.datasets import mnist
import numpy as np

autoencoder.fit(
    x_train,
    x_train,
    epochs=50,
    batch_size=256,
    shuffle=True,
    validation_data=(x_test, x_test),
)

# encode and decode some digits
# note that we take them from the *test* set
encoded_imgs = encoder.predict(x_test)
decoded_imgs = decoder.predict(encoded_imgs)